<a href="https://colab.research.google.com/github/MatP-DS/MasterThesis/blob/main/export_all_l8_bands_and_indices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import os
from google.colab import drive

# ------------------------------
# STEP 1: Setup
# ------------------------------
ee.Authenticate()
ee.Initialize(project='ee-masterthesisdeforestation')

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

aoi = ee.Geometry.Rectangle([-56.2, -11.8, -56.0, -11.6])


Mounted at /content/drive


In [ ]:
# ------------------------------
# STEP 2: Cloud masking for Landsat 8 SR
# ------------------------------
def maskL8sr(image):
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
           qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])


In [ ]:
# ------------------------------
# STEP 3: Add spectral indices
# ------------------------------
def add_all_indices(img):
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': img.select('SR_B5'), 'RED': img.select('SR_B4'), 'BLUE': img.select('SR_B2')}
    ).rename('EVI')
    ndwi = img.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    nbr = img.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')
    savi = img.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
        {'NIR': img.select('SR_B5'), 'RED': img.select('SR_B4')}
    ).rename('SAVI')

    return img.addBands([ndvi, evi, ndwi, nbr, savi])


In [ ]:
# ------------------------------
# STEP 4: Monthly export loop
# ------------------------------
output_folder = '02_landsat8/full_feature_stack'
local_check_dir = '/content/drive/MyDrive/MasterThesis/01_raw_data/' + output_folder
os.makedirs(local_check_dir, exist_ok=True)

years = [2020, 2021, 2022, 2023]

for year in years:
    for month in range(1, 13):
        start = ee.Date.fromYMD(year, month, 1)
        end = start.advance(1, 'month')
        filename = f'L8_full_{year}_{month:02d}.tif'
        local_path = os.path.join(local_check_dir, filename)

        if os.path.exists(local_path):
            print(f"⏭️ Skipping existing: {filename}")
            continue

        collection = (
            ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterDate(start, end)
            .filterBounds(aoi)
            .map(maskL8sr)
            .map(lambda img: img.multiply(0.0000275).add(-0.2).copyProperties(img, ["system:time_start"]))  # Scaling for SR bands
            .map(add_all_indices)
            .select([
                'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
                'NDVI', 'EVI', 'NDWI', 'NBR', 'SAVI'
            ])
        )

        composite = collection.mean().clip(aoi)

        if composite.bandNames().size().getInfo() == 0:
            print(f"⚠️ No data in {year}-{month:02d}")
            continue

        task = ee.batch.Export.image.toDrive(
            image=composite.toFloat(),
            description=f'Export_L8_full_{year}_{month:02d}',
            folder=output_folder.split("/")[-1],
            fileNamePrefix=f'L8_full_{year}_{month:02d}',
            region=aoi,
            scale=30,  # Landsat 8 native resolution
            fileFormat='GeoTIFF',
            maxPixels=1e13
        )
        task.start()
        print(f"🚀 Export started: {filename}")


⚠️ No data in 2020-01
🚀 Export started: L8_full_2020_02.tif
⚠️ No data in 2020-03
🚀 Export started: L8_full_2020_04.tif
🚀 Export started: L8_full_2020_05.tif
🚀 Export started: L8_full_2020_06.tif
🚀 Export started: L8_full_2020_07.tif
🚀 Export started: L8_full_2020_08.tif
🚀 Export started: L8_full_2020_09.tif
🚀 Export started: L8_full_2020_10.tif
🚀 Export started: L8_full_2020_11.tif
🚀 Export started: L8_full_2020_12.tif
🚀 Export started: L8_full_2021_01.tif
🚀 Export started: L8_full_2021_02.tif
🚀 Export started: L8_full_2021_03.tif
🚀 Export started: L8_full_2021_04.tif
🚀 Export started: L8_full_2021_05.tif
🚀 Export started: L8_full_2021_06.tif
🚀 Export started: L8_full_2021_07.tif
🚀 Export started: L8_full_2021_08.tif
🚀 Export started: L8_full_2021_09.tif
🚀 Export started: L8_full_2021_10.tif
🚀 Export started: L8_full_2021_11.tif
🚀 Export started: L8_full_2021_12.tif
🚀 Export started: L8_full_2022_01.tif
🚀 Export started: L8_full_2022_02.tif
🚀 Export started: L8_full_2022_03.tif
🚀 Expo